# Detectron2 (FAIR)


In [ ]:
import detectron2
import contextlib
import datetime
import io
import os
import json
import logging
import cv2
import random
import numpy as np
import copy,torch,torchvision
import PIL
from PIL import Image
import xml.etree.ElementTree as X
import math
from itertools import repeat

from detectron2.structures import Boxes, BoxMode, PolygonMasks
from detectron2.config import *
from detectron2.modeling import build_model
from detectron2 import model_zoo
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator,DatasetEvaluators, inference_on_dataset, coco_evaluation, DatasetEvaluator
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
import matplotlib.pyplot as plt

import glob
import time
import shutil
from multiprocessing.pool import ThreadPool
import concurrent.futures
# from pyunpack import Archive
from utility.LossEvalHook import *


import torch
torch.cuda.set_device(0)

setup_logger()

In [ ]:
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)


    # @classmethod
    # def build_train_loader(cls, cfg):
    #     dataloader = build_detection_train_loader(cfg,
    #         mapper=DatasetMapper(cfg, is_train=True, augmentations=[
    #         T.Resize((1000,1000)),
    #         T.RandomFlip(0.5, horizontal=True, vertical=False),
    #         T.RandomFlip(0.5, horizontal=False, vertical=True),
    #         T.RandomRotation(angle=[-60,60], expand=True, center=None, sample_style='range', interp=None),
    #         # T.RandomContrast(intensity_min=-1, intensity_max=2),
    #     ]))
    #     return dataloader   
        # use this dataloader instead of the default
        
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

## Dataset preperation

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
register_coco_instances("ship_dataset_train", {}, "ShipAnnotations/train2017.json", "ShipDataset")
register_coco_instances("ship_dataset_val", {}, "ShipAnnotations/test2017.json", "ShipDataset")

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
register_coco_instances("wake_dataset_train", {}, "Sar Wakes 3/train/coco.json", "Sar Wakes 3/train/imgs")
register_coco_instances("wake_dataset_val", {}, "Sar Wakes 3/validation/coco.json", "Sar Wakes 3/validation/imgs")

# MetadataCatalog.get("wake_dataset_train").set(thing_classes=["wake"])
# MetadataCatalog.get("wake_dataset_val").set(thing_classes=["wake"])

## Setup configuration
Now we've got out data in a usable form,and some useful functions lets configure our tests. below are the options for training, 

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = 'output'
#  Let training initialize from models.txt
MODEL = "Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(MODEL))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(MODEL) #
cfg.DATASETS.TRAIN = ("wake_dataset_train",)
cfg.DATASETS.TEST = ("wake_dataset_val",)

cfg.MODEL.MASK_ON=False
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # this is far lower than usual.  
cfg.MODEL.ROI_HEADS.NUM_CLASSES =  1             # only has one class.
cfg.MODEL.RETINANET.NUM_CLASSES =  1

cfg.SOLVER.IMS_PER_BATCH = 1          # can be up to  24 for a p100 (6 default)
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.TEST.EVAL_PERIOD = 500
cfg.SOLVER.BASE_LR = 0.05
cfg.SOLVER.GAMMA = 0.5
cfg.SOLVER.STEPS=[1000,1200,2500,3000]
cfg.SOLVER.MAX_ITER=5000

cfg.MODEL.PIXEL_MEAN = [60, 60, 60]
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True 
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)#lets just check our output dir exists

## Training
### Load tensorboard to view progress

In [ ]:
# # Look at training curves in tensorboard:
# %reload_ext tensorboard
# %tensorboard --logdir "output"

### Start training

In [ ]:
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
trainer = MyTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()

## Evaluation

The results produced about aren't as accurate as those performed outside of training. So let's rerun our evaluations. 

### COCO dataset conversions

In [ ]:
# Create coco evaluator, but use the default detectron2 data format for generation, make sure ids overlap
evaluator = COCOEvaluator("wake_dataset_val", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "wake_dataset_val") 
outputs = inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
cfg.MODEL.WEIGHTS = "checkpoints/Model Trained Wake/cascade_mask_rcnn_R_50_FPN_3x/model_0002979.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 
predictor = DefaultPredictor(cfg)




In [ ]:
# test_dict = get_dataset("wake_dataset_val")
from detectron2.utils.visualizer import ColorMode
test_img = glob.glob('Sar Wakes 3/validation/imgs/*.png')
for d in random.sample(test_img, 10):
  im = cv2.imread(d)
  outputs = predictor(im)  
  v = Visualizer(im[:, :, ::-1],
                  metadata=MetadataCatalog.get("wake_dataset_val"), 
                  scale=1)

 
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  plt.figure(figsize=(10,10))
  plt.imshow(out.get_image()[:, :, ::-1])
  plt.axis(False)
  plt.show()

In [ ]:
# test_dict = get_dataset("wake_dataset_val")
im = cv2.imread('ScenariTest/ShipWakeTerraSAR-X/VV.tiff')
outputs = predictor(im)  
v = Visualizer(im[:, :, ::-1],
                  metadata=MetadataCatalog.get("wake_dataset_val"), 
                  scale=1)
                  # instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
  # )
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize=(10,10))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis(False)
plt.show()

In [86]:
im = cv2.imread('ScenariTest/ShipWakeTerraSAR-X/VV.tiff')
x,y = im.shape[0], im.shape[1] 
range_x, range_y = np.arange(500, x-500, 500), np.arange(500, y-500, 500)

Bboxes = []
for space_x in range_x:
    for space_y in range_y:
        img = im[(space_x-500):(space_x+500),(space_y-500):(space_y+500)]
        outputs = predictor(img)
        bboxes=outputs["instances"].pred_boxes.tensor.cpu().numpy()
        for b in bboxes: 
            b[0], b[1], b[2], b[3] = 
            Bboxes.append(b)

Bboxes_arr = np.array(Bboxes)

In [97]:
Bboxes_arr = np.array(Bboxes)
Bboxes_arr

array([[4.03069611e+02, 8.07007568e+02, 9.93362549e+02, 9.83690491e+02],
       [4.44335022e+02, 8.14574463e+02, 9.02755737e+02, 9.93942383e+02],
       [1.02445984e+01, 7.78078552e+02, 8.85861816e+02, 9.79357239e+02],
       [0.00000000e+00, 8.10406738e+02, 4.23812012e+02, 9.93653015e+02],
       [6.90940186e+02, 7.35234314e+02, 9.91155640e+02, 9.14234253e+02],
       [5.62303406e+02, 4.94804718e+02, 9.93138123e+02, 9.80390625e+02],
       [1.18795090e+02, 3.10268890e+02, 9.75732361e+02, 4.74914429e+02],
       [4.56544304e+01, 4.75291504e+02, 5.29301636e+02, 9.90162720e+02],
       [3.09591766e+02, 3.00343018e+02, 9.42853088e+02, 5.98358826e+02],
       [2.03178024e+01, 2.82192017e+02, 8.74682495e+02, 4.78997955e+02],
       [2.40352249e+01, 2.65970612e+02, 9.43456421e+02, 5.76292053e+02],
       [7.26169586e+00, 3.00203796e+02, 5.66873291e+02, 5.11643616e+02],
       [6.93734070e+02, 2.36417694e+02, 9.86926331e+02, 4.18223206e+02],
       [2.20299713e+02, 5.69267700e+02, 9.08212158e

In [98]:
from utility.nms import non_max_suppression_fast

bboxes_draw = non_max_suppression_fast(Bboxes_arr, 0.1)


In [99]:
bboxes_draw

array([[265, 805, 768, 998],
       [ 67,  71, 900, 935]])